# La Commune de Paris 1871 en liberté - Faire une requête sur wikidata en Python

Ce chapitre documente un point technique important pour la suite. Il explique comment le code fonctionne, et n'interroge pas la pertinance des choix de la requête. Cela est vu à d'autres moment.

L'exemple est pris sur la recherche des communard·e·s présent·e·s dans Wikidata. L'explication de qu'est ce qu'un·e "communard·e·s présent·e·s dans Wikidata" sera vu plus loin.

### Méthode

La classe Extraction_wikidata, est une adaption du modèle de réquête en python qui se trouve sur https://query.wikidata.org/

- dans la variable query, il faut copier/coller la requête faite dans query.wikidata.org
- exécuter les cellules "## 0" à "## 5" comme si dessous afin d'obtenir le résultat dans un dataframe exploitable pour faire des manipulations de données.

Cela est reproductible, et l'est à de nombreuses occurence dans le document, sur plusieurs requêtes. 
Il suffit de changer la valeur de query (en ## 1) puis personnaliser les ##4 et 5 pour obtenir un tableau (dataframe) exploitable.

In [2]:
# les installations nécessaires
#https://rdflib.github.io/sparqlwrapper/
#!pip install sparqlwrapper
#!pip install seaborn

In [3]:
## 0 - import
import sys

from SPARQLWrapper import SPARQLWrapper, JSON

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from IPython.display import HTML

In [4]:
## 1 - les variables
endpoint_url:str = "https://query.wikidata.org/sparql"

In [5]:
query:str = """SELECT ?communard_ou_communarde ?communard_ou_communardeLabel WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],fr". }
  ?communard_ou_communarde wdt:P106 wd:Q1780490.
}"""

In [6]:
## 2 - l'objet
class Extraction_wikidata():
    def __init__(self, endpoint_url:str, query:str):
        self.endpoint_url:str = endpoint_url
        self.query:str = query
        
    def get_results(self, endpoint_url:str, query:str):
        user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
        # TODO adjust user agent; see https://w.wiki/CX6
        sparql = SPARQLWrapper(endpoint_url, agent = user_agent)
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
        return sparql.query().convert()
    
    def json_to_df(self, entree):
        #results = sparql.query().convert()
        df_results = pd.json_normalize(entree["results"]["bindings"])
        return df_results
    
    def extraire_et_df(self):
        extract_json = self.get_results(self.endpoint_url, self.query)
        extract_df = self.json_to_df(extract_json)
        return extract_df


In [7]:
## 3 - instantiation de l'objet et récupération du df
extraction_communard = Extraction_wikidata(endpoint_url, query)
df_communard = extraction_communard.extraire_et_df()

In [8]:
## 4 - afficher le df voulu
print(df_communard)

    communard_ou_communarde.type              communard_ou_communarde.value  \
0                            uri      http://www.wikidata.org/entity/Q20951   
1                            uri      http://www.wikidata.org/entity/Q34618   
2                            uri     http://www.wikidata.org/entity/Q184535   
3                            uri     http://www.wikidata.org/entity/Q216092   
4                            uri     http://www.wikidata.org/entity/Q257950   
..                           ...                                        ...   
305                          uri  http://www.wikidata.org/entity/Q115257385   
306                          uri  http://www.wikidata.org/entity/Q115257386   
307                          uri  http://www.wikidata.org/entity/Q115257387   
308                          uri  http://www.wikidata.org/entity/Q115257388   
309                          uri  http://www.wikidata.org/entity/Q118198782   

    communard_ou_communardeLabel.xml:lang communard

In [9]:
## 5 - quelques statistiques sur le df
#df_communard.describe()

In [10]:
df_communard_en_html = df_communard.to_html(columns = ['communard_ou_communarde.value', 'communard_ou_communardeLabel.value'])
HTML(df_communard_en_html)

,communard_ou_communarde.value,communard_ou_communardeLabel.value
0,http://www.wikidata.org/entity/Q20951,Élisée Reclus
1,http://www.wikidata.org/entity/Q34618,Gustave Courbet
2,http://www.wikidata.org/entity/Q184535,Sofia Kovalevskaïa
3,http://www.wikidata.org/entity/Q216092,Louise Michel
4,http://www.wikidata.org/entity/Q257950,Auguste-Jean-Marie Vermorel
5,http://www.wikidata.org/entity/Q274464,Édouard Vaillant
6,http://www.wikidata.org/entity/Q309722,Auguste Blanqui
7,http://www.wikidata.org/entity/Q312915,Eugène Pottier
8,http://www.wikidata.org/entity/Q337628,Piotr Lavrov
9,http://www.wikidata.org/entity/Q349485,Jules Allix


### Avoir TOUTES les informations sur les communard·e·s

Cette requête, qui fonctionne comme expliqué ci dessus est importante. Car en plus de charger toutes les personnes concernées (leur label) elle récupére toutes les données liées à leur élément.

Ce dataframe sera ré-utilisé à de nombreuses occasions.

Pour toutes les personnes, chercher les ?p ?q avec gestion des labels.
Attention : c'est lourd !

#### Aller chercher les valeurs et les mettre dans un dataframe (df) nommé df_communard_tous_p_q

In [11]:
query = """SELECT ?communard_ou_communarde ?communard_ou_communardeLabel ?p ?pLabel ?q ?qLabel WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "fr,en".
                         ?communard_ou_communarde rdfs:label ?communard_ou_communardeLabel.
                         ?p rdfs:label ?pLabel.
                         ?q rdfs:label ?qLabel.}
  
  ?communard_ou_communarde wdt:P106 wd:Q1780490;
                           ?p ?q. 
}
"""

In [12]:
extraction_communard_tous_p_q = Extraction_wikidata(endpoint_url, query)
df_communard_tous_p_q = extraction_communard_tous_p_q.extraire_et_df()

In [13]:
df_communard_tous_p_q

,communard_ou_communarde.type,communard_ou_communarde.value,p.type,p.value,q.datatype,q.type,q.value,communard_ou_communardeLabel.xml:lang,communard_ou_communardeLabel.type,communard_ou_communardeLabel.value,pLabel.type,pLabel.value,qLabel.type,qLabel.value,q.xml:lang,qLabel.xml:lang
0,uri,http://www.wikidata.org/entity/Q20951,uri,http://schema.org/version,http://www.w3.org/2001/XMLSchema#integer,literal,2264324905,fr,literal,Élisée Reclus,literal,http://schema.org/version,literal,2264324905,NaN,NaN
1,uri,http://www.wikidata.org/entity/Q20951,uri,http://schema.org/dateModified,http://www.w3.org/2001/XMLSchema#dateTime,literal,2024-10-23T21:40:18Z,fr,literal,Élisée Reclus,literal,http://schema.org/dateModified,literal,2024-10-23T21:40:18Z,NaN,NaN
2,uri,http://www.wikidata.org/entity/Q20951,uri,http://schema.org/description,NaN,literal,ލިޔުންތެރިއެއް,fr,literal,Élisée Reclus,literal,http://schema.org/description,literal,ލިޔުންތެރިއެއް,dv,NaN
3,uri,http://www.wikidata.org/entity/Q20951,uri,http://schema.org/description,NaN,literal,France karimba ŋun nyɛ doo,fr,literal,Élisée Reclus,literal,http://schema.org/description,literal,France karimba ŋun nyɛ doo,dag,NaN
4,uri,http://www.wikidata.org/entity/Q20951,uri,http://schema.org/description,NaN,literal,francouzský geograf a anarchista,fr,literal,Élisée Reclus,literal,http://schema.org/description,literal,francouzský geograf a anarchista,cs,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34467,uri,http://www.wikidata.org/entity/Q3083515,uri,http://www.wikidata.org/prop/P1412,NaN,uri,http://www.wikidata.org/entity/statement/Q3083...,fr,literal,François-Louis Parisel,literal,http://www.wikidata.org/prop/P1412,literal,statement/Q3083515-5A31CF4A-B593-496C-A271-CEA...,NaN,NaN
34468,uri,http://www.wikidata.org/entity/Q3083515,uri,http://www.wikidata.org/prop/P1559,NaN,uri,http://www.wikidata.org/entity/statement/Q3083...,fr,literal,François-Louis Parisel,literal,http://www.wikidata.org/prop/P1559,literal,statement/Q3083515-6F000457-A1A2-45B7-9209-873...,NaN,NaN
34469,uri,http://www.wikidata.org/entity/Q3083515,uri,http://www.wikidata.org/prop/P1596,NaN,uri,http://www.wikidata.org/entity/statement/Q3083...,fr,literal,François-Louis Parisel,literal,http://www.wikidata.org/prop/P1596,literal,statement/Q3083515-34979a69-4553-0922-956c-0f4...,NaN,NaN
34470,uri,http://www.wikidata.org/entity/Q3083515,uri,http://www.wikidata.org/prop/P2671,NaN,uri,http://www.wikidata.org/entity/statement/Q3083...,fr,literal,François-Louis Parisel,literal,http://www.wikidata.org/prop/P2671,literal,statement/Q3083515-D5DCA60C-88FE-4EC8-8F6D-EDE...,NaN,NaN


#### Quelques apperçu de travail sur ce df

In [14]:
#df_communard_tous_p_q.describe()

In [15]:
df_communard_tous_p_q.columns

Index(['communard_ou_communarde.type', 'communard_ou_communarde.value',
       'p.type', 'p.value', 'q.datatype', 'q.type', 'q.value',
       'communard_ou_communardeLabel.xml:lang',
       'communard_ou_communardeLabel.type',
       'communard_ou_communardeLabel.value', 'pLabel.type', 'pLabel.value',
       'qLabel.type', 'qLabel.value', 'q.xml:lang', 'qLabel.xml:lang'],
      dtype='object')

In [16]:
df_communard_tous_p_q.tail(50)

,communard_ou_communarde.type,communard_ou_communarde.value,p.type,p.value,q.datatype,q.type,q.value,communard_ou_communardeLabel.xml:lang,communard_ou_communardeLabel.type,communard_ou_communardeLabel.value,pLabel.type,pLabel.value,qLabel.type,qLabel.value,q.xml:lang,qLabel.xml:lang
34422,uri,http://www.wikidata.org/entity/Q3083515,uri,http://www.wikidata.org/prop/direct/P19,NaN,uri,http://www.wikidata.org/entity/Q3209735,fr,literal,François-Louis Parisel,literal,http://www.wikidata.org/prop/direct/P19,literal,La Guillotière,NaN,fr
34423,uri,http://www.wikidata.org/entity/Q3083515,uri,http://www.wikidata.org/prop/direct/P20,NaN,uri,http://www.wikidata.org/entity/Q25395,fr,literal,François-Louis Parisel,literal,http://www.wikidata.org/prop/direct/P20,literal,Newark,NaN,fr
34424,uri,http://www.wikidata.org/entity/Q3083515,uri,http://www.wikidata.org/prop/direct/P21,NaN,uri,http://www.wikidata.org/entity/Q6581097,fr,literal,François-Louis Parisel,literal,http://www.wikidata.org/prop/direct/P21,literal,masculin,NaN,fr
34425,uri,http://www.wikidata.org/entity/Q3083515,uri,http://www.wikidata.org/prop/direct/P27,NaN,uri,http://www.wikidata.org/entity/Q142,fr,literal,François-Louis Parisel,literal,http://www.wikidata.org/prop/direct/P27,literal,France,NaN,fr
34426,uri,http://www.wikidata.org/entity/Q3083515,uri,http://www.wikidata.org/prop/direct/P31,NaN,uri,http://www.wikidata.org/entity/Q5,fr,literal,François-Louis Parisel,literal,http://www.wikidata.org/prop/direct/P31,literal,être humain,NaN,fr
34427,uri,http://www.wikidata.org/entity/Q3083515,uri,http://www.wikidata.org/prop/direct/P69,NaN,uri,http://www.wikidata.org/entity/Q3064295,fr,literal,François-Louis Parisel,literal,http://www.wikidata.org/prop/direct/P69,literal,faculté de pharmacie de Paris,NaN,fr
34428,uri,http://www.wikidata.org/entity/Q3083515,uri,http://www.wikidata.org/prop/direct/P106,NaN,uri,http://www.wikidata.org/entity/Q39631,fr,literal,François-Louis Parisel,literal,http://www.wikidata.org/prop/direct/P106,literal,médecin,NaN,fr
34429,uri,http://www.wikidata.org/entity/Q3083515,uri,http://www.wikidata.org/prop/direct/P106,NaN,uri,http://www.wikidata.org/entity/Q1780490,fr,literal,François-Louis Parisel,literal,http://www.wikidata.org/prop/direct/P106,literal,communard ou communarde,NaN,fr
34430,uri,http://www.wikidata.org/entity/Q3083515,uri,http://www.wikidata.org/prop/direct/P214,NaN,literal,259146284394915331471,fr,literal,François-Louis Parisel,literal,http://www.wikidata.org/prop/direct/P214,literal,259146284394915331471,NaN,NaN
34431,uri,http://www.wikidata.org/entity/Q3083515,uri,http://www.wikidata.org/prop/direct-normalized...,NaN,uri,http://viaf.org/viaf/259146284394915331471,fr,literal,François-Louis Parisel,literal,http://www.wikidata.org/prop/direct-normalized...,literal,http://viaf.org/viaf/259146284394915331471,NaN,NaN


#### Nettoyer le df et le mettre dans df_tout_sur_tous_communard

In [17]:
# retirer les colonnes "inutiles"
df_nettoyage_colonne = df_communard_tous_p_q.loc[:,['communard_ou_communarde.value','p.value','q.value','communard_ou_communardeLabel.xml:lang','communard_ou_communardeLabel.value', 'pLabel.value','qLabel.value', 'q.xml:lang']]
# garder le français
df_nettoyage_langue = df_nettoyage_colonne[df_nettoyage_colonne['q.xml:lang'] == ("fr" or "NaN")]

In [18]:
df_nettoyage_langue.describe()

,communard_ou_communarde.value,p.value,q.value,communard_ou_communardeLabel.xml:lang,communard_ou_communardeLabel.value,pLabel.value,qLabel.value,q.xml:lang
count,1074,1074,1074,1074,1074,1074,1074,1074
unique,310,10,774,1,310,10,774,1
top,http://www.wikidata.org/entity/Q309722,http://www.w3.org/2000/01/rdf-schema#label,communarde,fr,Auguste Blanqui,http://www.w3.org/2000/01/rdf-schema#label,communarde,fr
freq,11,310,47,1074,11,310,47,1074


In [19]:
# Avoir un df exploitable pour la suite
df_tout_sur_tous_communard = df_nettoyage_langue

## fichiers pythons
Ces scripts sont ré-écrits dans les fichiers :
- extraction_wd.py pour faire une requete dans wikidata et obtenir un dataframe à exploiter en Python.
- extraction_communard1.py pour chercher les informations sur les communard, les mettres dans un dataframe et les analyser en Python

## Licence

Ce travail est sous licence libre plus précisement Creative Commons Attribution - Partage dans les Mêmes Conditions 4.0 International.

Il est fait à partir de données libres (diverse licence) et logiciels ou formats libres.
Citons python, notebook, makedon, mediawiki...
J'autorise la diffusion commerciale pour payer les frais de papier si quelqu'an veut le diffuser en version papier.
Par principe, la licence vous autorise à utiliser ce travail sans m'en informer, juste pour ma curiosité, merci de me le signaler.

<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Licence Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a><br />Cette œuvre est mise à disposition selon les termes de la <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Licence Creative Commons Attribution -  Partage dans les Mêmes Conditions 4.0 International</a>.

La version originale du notebook et des fichiers liées sont versionnés et héberger sur github. 
https://github.com/silanoc/commune_dans_wikidata